In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import subprocess, cv2, json, os, sys, shutil, pyk4a, time
import numpy as np, matplotlib.pyplot as plt
from kinectacq.acquisition import start_recording
from kinectacq.paths import DATA_DIR, ensure_dir

### Set up recording location and duration info

In [3]:
import datetime

In [4]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
timestamp

'2022-02-17_16-50-14'

In [5]:
filename_prefix = DATA_DIR / 'test_recording' / timestamp
recording_length = 10

In [6]:
ensure_dir(filename_prefix)

In [7]:
filename_prefix

PosixPath('/n/groups/datta/tim_sainburg/projects/kinectacq/data/test_recording/2022-02-17_16-50-14')

### Get camera information

In [8]:
!k4arecorder --list

Index:0	Serial:000774310512	Color:1.6.102	Depth:1.6.75
Index:1	Serial:000261501812	Color:1.6.108	Depth:1.6.79


In [9]:
import datetime, subprocess, numpy as np, cv2, time, sys
from multiprocessing import Process, Queue
from pyk4a import *


def capture_from_azure(k4a, filename_prefix):
        
    k4a.start()
    
    
    print('started: {}'.format(filename_prefix.split('.')[-1]))
    k4a.stop()

In [10]:
filename_prefix = 'data/222_2_2_GRIN61_MeA8A_training'
recording_length = 10

filename_prefix += '.rig2'

In [11]:
ensure_dir(filename_prefix)

In [12]:
devices = {
    "master": {
        "id": 0,
        "pyk4a_config": {
            "color_resolution": ColorResolution.RES_720P,
            "depth_mode": DepthMode.NFOV_UNBINNED,
            #"synchronized_images_only": False,
            "wired_sync_mode": WiredSyncMode.MASTER,
        },
        "process_kwargs": {
            "display_frames": True,
            "display_time": False,
        },
    },
    "subordinate": {
        "id": 1,
        "pyk4a_config": {
            "color_resolution": ColorResolution.RES_720P,
            "depth_mode": DepthMode.NFOV_UNBINNED,
            #"synchronized_images_only": False,
            "wired_sync_mode": WiredSyncMode.SUBORDINATE,
        },
        "process_kwargs": {
            "display_frames": False,
            "display_time": False,
        },
    }
}

In [13]:
process_list = []
for device_name in devices:

    # Create a k4a object referencing master
    k4a_obj = PyK4A(
        Config(**devices[device_name]["pyk4a_config"]),
        device_id=devices[device_name]["id"],
    )
    # create a subprocess to run acqusition with that camera
    process_list.append(
        Process(
            target=capture_from_azure,
            args=(
                k4a_obj,
                filename_prefix + '.{}'.format(device_name),
            ),
        )
    )

In [14]:
for p in process_list:
    p.start()

started: master
started: subordinate
